<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2025/blob/main/rhs2tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift, rhs2TF

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import sympy
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root

In [3]:
q = 100 #L/min
cai = 1. #mol/L
Ti = 350 #K
V = 100 #L
rho = 1000 #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750 #K
k0 = 7.2e10 #1/min
UA = 5e4 #J/(min K)
Tc0 = 300 #K
Tc = 300 #K
ca0 = 0.5 #mol/L
T0 = 350 #K


In [4]:
def rhs(t, x, u):
    cai, Ti, Tc = u
    ca, T = x
    k = k0*jnp.exp(-ER/T)
    return [q/V * (cai - ca) - k*ca,
            q/V * (Ti - T) + negHr*k*ca/(rho*C) - UA/(rho*V*C)*(T-Tc)]

In [5]:
u0=jnp.array([cai, Ti, Tc0])
x0=jnp.array(root(lambda x: rhs(0., x, [cai, Ti, Tc0]), [ca0,T0]).x)

In [7]:
rhs2TF(rhs,x0,u0)

Matrix([
[(s - 4.380542671477)/(s**2 - 2.38021576189*s - 1.287481641415),       -0.03571899397/(s**2 - 2.38021576189*s - 1.287481641415),                    -0.074725928807/(s**2 - 2.38021576189*s - 1.287481641415)],
[    209.273412047532/(s**2 - 2.38021576189*s - 1.287481641415), (s + 2.000326909587)/(s**2 - 2.38021576189*s - 1.287481641415), (2.092050209205*s + 4.18478432968)/(s**2 - 2.38021576189*s - 1.287481641415)]])

In [38]:
G01=rhs2TF(rhs,x0,u0, ios=(0,1))[0]
G00=rhs2TF(rhs,x0,u0, ios=(0,0))[0]

In [40]:
tsim, ysim1 = sim(G01, lambda t: 1, N=100, dt= 0.01)
tsim, ysim0 = sim(G00, lambda t: 0.1, N=100, dt= 0.01)

In [41]:
res=solve_ivp(lambda t,x: rhs(t,x,[cai+0.1, Ti+1, Tc0]), (0,tsim[-1]), x0, method='Radau', dense_output=True)

In [42]:
fig=make_subplots()
fig.add_scatter(x=tsim, y=ysim1+ysim0 + x0[0], name='Ca_TF')
fig.add_scatter(x=tsim, y=res.sol(tsim)[0], name='Ca_solve_ivp')
fig.update_layout(width=800, height=400, template='plotly_dark')

In [12]:
tend = 10 #min
res=solve_ivp(lambda t,x: rhs(t,x, [cai+0.1, Ti+5, Tc0+5]), (0,tend), x0, method='Radau', dense_output=True)

In [13]:
tplot=np.linspace(0,tend,200)
ca, T = res.sol(tplot)
fig=make_subplots(rows=1, cols=2)
fig.add_scatter(x=tplot, y=T, row=1, col=1, name='T')
fig.add_scatter(x=tplot, y=ca, row=1, col=2, name='Ca')
fig.update_layout(width=800, height=400, template='plotly_dark')

In [14]:
G=1/(s+5)**8

In [15]:
n,d = sympy.fraction(G)

In [16]:
a=sympy.Poly(d).all_coeffs()

In [17]:
sp.signal.residue(1., a)

(array([ 1409167.79588093+7.73552235e+00j,
          959512.05712454+1.01163772e+06j,
          959524.58585648-1.01299923e+06j,
          -52545.67544925+1.35699022e+06j,
          -51092.78836315-1.35699622e+06j,
         -959537.690768  +9.08318689e+05j,
         -959543.97506693-9.09209221e+05j,
        -1304544.23629199+4.57949488e+00j]),
 array([-4.90178796+0.j        , -4.93012428+0.06926201j,
        -4.93012428-0.06926201j, -4.99937   +0.09881876j,
        -4.99937   -0.09881876j, -5.0698751 +0.07052263j,
        -5.0698751 -0.07052263j, -5.0994733 +0.j        ]),
 array([], dtype=float64))